In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 10

## 11.1. 날짜, 시간자료형, 도구  
datetime,time,calande모듈

In [2]:
from datetime import datetime
now = datetime.now()
now
now.year,now.month,now.day

datetime.datetime(2019, 9, 1, 14, 42, 37, 311080)

(2019, 9, 1)

datetime은 날짜와 시간을 모두 저장하며, 마이크로초까지 지원함

In [3]:
now.second;now.microsecond
now.microsecond//1000;now.microsecond%1000

37

311080

311

80

datetime.timedelta는 두 객체간 시간적 차이를 표현가능

In [4]:
delta = datetime(2011,1,7) - datetime(2008,6,24,8,15)
delta
delta.days
delta.seconds

datetime.timedelta(days=926, seconds=56700)

926

56700

time delta를 더하거나 빼면 그만큼의 시간이 datetime객체에 적용되어 새 객체를 만듦

In [5]:
from datetime import timedelta
start = datetime(2011,1,7)
start
start + timedelta(12)#일,초,마이크로초
start - 2* timedelta(2)

datetime.datetime(2011, 1, 7, 0, 0)

datetime.datetime(2011, 1, 19, 0, 0)

datetime.datetime(2011, 1, 3, 0, 0)

### 11.1.1 문자열을 datetime으로 변환하기  
포맷규칙을 넘겨서 문자열로 나타낼 수 있음

In [6]:
stamp = datetime(2011,1,3)
stamp
str(stamp)
stamp.strftime('%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

'2011-01-03 00:00:00'

'2011-01-03'

In [7]:
#문자열을 날짜로 변환할 수도 있음
value = '2011-01-03'
datetime.strptime(value,'%Y-%m-%d')

datestrs = ['7/6/2011','8/6/2011']
[datetime.strptime(value,'%d/%m/%Y') for value in datestrs]

datetime.datetime(2011, 1, 3, 0, 0)

[datetime.datetime(2011, 6, 7, 0, 0), datetime.datetime(2011, 6, 8, 0, 0)]

datetime.strptime을 사용하는것이 강력하긴 하지만 매번 규칙을 명시하는것은 귀찮음..  
dateutil에 parse.parse메서드를 쓰면 편리함 (Pandas와 함께 설치됨)

In [8]:
from dateutil.parser import parse

parse('2011-06-07')
parse('jAn 31,2019 11:40')
'''사람이 인지하는 거의 대부분의 표현방식을 파싱가능'''

datetime.datetime(2011, 6, 7, 0, 0)

datetime.datetime(2019, 1, 31, 11, 40)

'사람이 인지하는 거의 대부분의 표현방식을 파싱가능'

In [9]:
#날짜가 월 앞에 오는경우 dayfirst=True
parse('6/12/2011',dayfirst = True)

datetime.datetime(2011, 12, 6, 0, 0)

In [10]:
#to_datetime
datestrs = ['2011-07-06 12:00:00','2011-09-06 12:00:00']
pd.to_datetime(datestrs)

#누락된값으로 간주해야할 값도 처리해줌
idx = pd.to_datetime(datestrs + [None])
idx
pd.isnull(idx) #Not a Time

DatetimeIndex(['2011-07-06 12:00:00', '2011-09-06 12:00:00'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2011-07-06 12:00:00', '2011-09-06 12:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

array([False, False,  True])

## 11.2. 시계열 기초  
가장 기본적인 시계열 객체는 문자열이나 datetime객체로 표현되는 타임스탬프 Series

In [11]:
from datetime import datetime
dates = [datetime(2011,1,2),datetime(2011,1,5),
        datetime(2011,1,7),datetime(2011,1,8),
        datetime(2011,1,10),datetime(2011,1,12)]
ts = pd.Series(np.random.randn(6),index = dates)
ts
ts.index

2011-01-02    1.131501
2011-01-05   -1.238637
2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

numpy의 datetime64를 사용해 나노초 단위로 타임스탬프 저장  
DatetimeIndex의 스칼라값은 Timestamp 객체

In [12]:
ts.index.dtype
ts.index[0]

dtype('<M8[ns]')

Timestamp('2011-01-02 00:00:00')

In [13]:
ts + ts[::2]

2011-01-02    2.263001
2011-01-05         NaN
2011-01-07    0.009274
2011-01-08         NaN
2011-01-10    2.266186
2011-01-12         NaN
dtype: float64

### 11.2.1 색인, 선택, 부분선택

시계열의 인덱싱은 시리즈와 동일함

In [14]:
ts
stamp = ts.index[2]
ts[stamp]

2011-01-02    1.131501
2011-01-05   -1.238637
2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

0.004637037003246075

해석가능한 문자를 넘겨서 편리하게 사용할 수 있음

In [15]:
ts['1/7/2011']
ts['20110107']

0.004637037003246075

0.004637037003246075

긴 시계열에서 연이나 월만 넘겨서 데이터 일부를 선택할수도있음

In [16]:
longer_ts = pd.Series(np.random.randn(1000),
                     index = pd.date_range('1/1/2000',periods=1000))
longer_ts
longer_ts['2001']

2000-01-01   -1.133166
2000-01-02   -1.572344
2000-01-03   -0.384777
2000-01-04    0.951442
2000-01-05   -0.477794
                ...   
2002-09-22    2.202454
2002-09-23   -0.804767
2002-09-24   -0.084637
2002-09-25   -1.762638
2002-09-26    1.169230
Freq: D, Length: 1000, dtype: float64

2001-01-01   -0.348587
2001-01-02   -0.477844
2001-01-03   -0.946755
2001-01-04   -0.153351
2001-01-05    0.270694
                ...   
2001-12-27    2.423387
2001-12-28    0.065935
2001-12-29    0.097193
2001-12-30   -1.520763
2001-12-31   -0.461759
Freq: D, Length: 365, dtype: float64

In [17]:
longer_ts['2001-05']

2001-05-01    0.520532
2001-05-02    0.062845
2001-05-03    1.086768
2001-05-04    0.478732
2001-05-05   -1.066658
                ...   
2001-05-27   -1.663930
2001-05-28   -0.494743
2001-05-29   -0.766146
2001-05-30   -1.723497
2001-05-31   -0.881968
Freq: D, Length: 31, dtype: float64

슬라이싱도 유사함

In [18]:
ts[datetime(2011,1,7):]

2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

In [19]:
ts
ts['1/6/2011':'01/11/2011']

2011-01-02    1.131501
2011-01-05   -1.238637
2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
dtype: float64

슬라이스된 데이터는 원본의 뷰이므로 변경사항이 원본에 적용된다는것에 유의!

truncate 인스턴스 메서드는 TimeSeries를 두개의 날짜로 나눔

In [20]:
ts
ts.truncate(before='1/9/2011')
ts.truncate(after='1/9/2011')

2011-01-02    1.131501
2011-01-05   -1.238637
2011-01-07    0.004637
2011-01-08    0.877150
2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

2011-01-10    1.133093
2011-01-12    0.277973
dtype: float64

2011-01-02    1.131501
2011-01-05   -1.238637
2011-01-07    0.004637
2011-01-08    0.877150
dtype: float64

**DataFrame에도 동일하게 적용되며, 로우에 인덱싱됨**

In [21]:
dates = pd.date_range('1/1/2000',periods = 100,freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100,4),
                      index = dates,
                      columns = ['Colorado','Texas','New York','Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.882055,0.865215,-2.238999,-0.885110
2001-05-09,-0.871110,0.468433,-1.347181,0.332800
2001-05-16,0.360573,2.202293,0.402414,1.146535
2001-05-23,-2.862731,-0.317551,-0.012387,0.010088
2001-05-30,-1.552841,-0.138858,-0.423759,0.442692


### 11.2.2. 중복된 색인을 갖는 시계열

In [22]:
dates = pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dup_ts = pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

is_unique 속성을 보면 색인이 유일하지 않음을 확인가능

In [23]:
dup_ts.index.duplicated()

array([False, False,  True,  True, False])

In [24]:
dup_ts.index.is_unique

False

이걸 인덱싱하면 중복여부에 따라 스칼라 혹은 슬라이스가 생성

In [25]:
dup_ts['1/3/2000']
dup_ts['1/2/2000']

4

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

유일하지 않은 타임스탬프를 가진 데이터를 집계한다고 하면, 한가지 방법은 groupby에 level=0(단일단계 인덱싱)을 넘기는것

In [26]:
# dup_ts.groupby(dup_ts.index).mean()
dup_ts.groupby(level=0).mean()
dup_ts.groupby(level=0).count()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64